# Use of PSF simulations code

In [ ]:
from simu_PSF_polar import *
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.colors import LogNorm

from tqdm import tqdm

if you feed scalars, all the computation is performed with numpy, otherwise it uses pyTorch. The goal is that when using torch, all the variables are stored as tensors in the good device (cpu or gpu). 

In [ ]:
version = 'numpy'
#version = 'torch'
device = 'cpu'

In [ ]:
N_photons=1000 # number of photons collected 
N=100 # discretization of the BFP
l_pixel=4 #pixel size in micrometer
NA=1.4 # numerical aperture
mag=100 # first magnification
lambd=617 # wavelength
f_tube=200 # tube lens focal
MAG=200/150 # second magnification

if version == 'torch':
    N_photons=torch.tensor(N_photons, device=device)
    N=torch.tensor(N, device=device)
    l_pixel=torch.tensor(l_pixel, device=device)
    NA=torch.tensor(NA, device=device)
    mag=torch.tensor(mag, device=device)
    lambd=torch.tensor(lambd, device=device)
    f_tube=torch.tensor(f_tube, device=device)
    MAG=torch.tensor(MAG, device=device)

In [ ]:
xp = 0. # dipole position
yp = 0. # dipole position
z = 3.*0.617 # z position of dipole
d_ = -3.*0.617/0.8 # defocus of dipole
rho = 70 # equatorial angle in degree
eta = 100 # azimuthal angle in degree
delta = 180 # wobbling in degree

In [ ]:
# this parameter allows to give a tensor of several inputs (the same number for all inputs)
# the PSF computation is parallelized and the output has one more dimension (the first)
several_psf = True
several_planes = False

if several_psf and version=='torch':
    xp = torch.tensor([xp for k in range(10)], device=device)
    yp = torch.tensor([yp for k in range(10)], device=device)
    z = torch.tensor([z for k in range(10)], device=device)
    d_ = torch.tensor([d_ for k in range(10)], device=device)
    rho = torch.tensor([rho for k in range(10)], device=device)
    eta = torch.tensor([eta for k in range(10)], device=device)
    delta = torch.tensor([delta for k in range(10)], device=device)

elif not several_psf and version=='torch':
    xp=torch.tensor(xp, device=device)
    yp=torch.tensor(yp, device=device)
    z=torch.tensor(z, device=device)
    d_=torch.tensor(d_, device=device)
    rho=torch.tensor(rho, device=device)
    eta=torch.tensor(eta, device=device)
    delta=torch.tensor(delta, device=device)

In [ ]:
# this functions essentially computes the BFP fields in the case of perfect focus emmited by a dpiole along x/y/z and projected on polarizations x/y
# x, y are meshgrids representing the BFP, distances renormalized by f_tube
# th1 and phi are the angles in the BFP
# r is the radius/f_tube in the BFP
# [Ex0, Ex1, Ex2] BFP fields from x/y/z dipoles projected on x
# [Ey0, Ey1, Ey2] BFP fields from x/y/z dipoles projected on y
# f_o is the objective focal to match the magnification

# THIS ONLY DEPENDS ON THE MICROSCOPE PARAMETERS AND IS GENERALLY COMPUTED ONCE ONLY

In [ ]:
x, y, th1, phi, [Ex0, Ex1, Ex2], [Ey0, Ey1, Ey2], r, r_cut, k, f_o = vectorial_BFP_perfect_focus(N, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, device='cpu')

In [ ]:
rho = rho * np.pi / 180
eta = eta * np.pi / 180
delta = delta * np.pi / 180

In [ ]:
R = np.array([
            [(np.sin(rho)**2)*(1 - np.cos(eta)) + np.cos(eta), 
             np.sin(rho) * np.cos(rho) * (np.cos(eta) - 1), 
             np.cos(rho) * np.sin(eta)],
            [np.sin(rho) * np.cos(rho) * (np.cos(eta) - 1), 
             (np.cos(rho)**2)*(1 - np.cos(eta)) + np.cos(eta), 
             np.sin(rho) * np.sin(eta)],
            [-np.cos(rho) * np.sin(eta), 
             -np.sin(rho) * np.sin(eta), 
             np.cos(eta)]
        ])

In [ ]:
mu = np.array([np.sin(eta)*np.cos(rho), np.sin(eta)*np.sin(rho), np.cos(eta)])

In [ ]:
lam = np.array([
            (1 - np.cos(delta / 2)) * (np.cos(delta / 2) + 2) / 6, 
            (1 - np.cos(delta / 2)) * (np.cos(delta / 2) + 2) / 6, 
            ((np.cos(delta / 2)**3 - 1) / (np.cos(delta / 2) - 1)) / 3
        ])

In [ ]:
zer = np.zeros(Ex0.shape)

In [ ]:
Ex = np.array([Ex0, Ex1, Ex2])
Ey = np.array([Ey0, Ey1, Ey2])
#Ex = np.array([zer, Ex1, zer])
#Ey = np.array([zer, zer, zer])

r_cut = 1.4/1.518
N=100
x, y = np.meshgrid(np.linspace(-r_cut,r_cut,N), np.linspace(-r_cut,r_cut,N))
r = np.sqrt(x**2+y**2)

th1 = np.zeros((N,N))
th1[:] = np.nan
th1[r<r_cut] = np.arcsin(r[r<r_cut])
phi = np.arctan2(y,x)

Ex = np.array([np.sin(phi)**2+(np.cos(phi)**2)*np.sqrt(1-np.sin(th1)**2), np.sin(2*phi)*(np.sqrt(1-np.sin(th1)**2)-1)/2, -np.sin(th1)*np.cos(phi)])
Ey = np.array([np.sin(2*phi)*(np.sqrt(1-np.sin(th1)**2)-1)/2, np.cos(phi)**2+(np.sin(phi)**2)*np.sqrt(1-np.sin(th1)**2), -np.sin(th1)*np.sin(phi)])
for uuu in [0,1,2]:
    Ex[uuu][np.isnan(Ex[uuu])]=0.
    Ey[uuu][np.isnan(Ey[uuu])]=0.

In [ ]:
full_bfpx = np.zeros(Ex[0].shape)

first_sum = np.einsum('ab, auv -> buv', R, Ex)
full_bfpx = np.einsum('a, auv -> uv', lam, np.conj(first_sum)*first_sum)

first_sum = np.einsum('ab, auv -> buv', R, Ey)
full_bfpy = np.einsum('a, auv -> uv', lam, np.conj(first_sum)*first_sum)

full_bfpx = full_bfpx
full_bfpy = full_bfpy

In [ ]:
bfp_field = np.sqrt((full_bfpx+full_bfpy))
bfp_field[np.isnan(bfp_field)] = 0.

In [ ]:
f_tube = 200000
f_o = f_tube/100
Dx = 2*r_cut*f_o/N
l_pixel=4
k = 2*n1*np.pi/0.617
Npadding = int((2*np.pi*f_tube)/(k*l_pixel*Dx)) - N
if Npadding%2==1:
    Npadding=Npadding+1

print(Npadding)
bfp_field_ = np.zeros((N+Npadding, N+Npadding), dtype='float64')
bfp_field_[Npadding//2:Npadding//2+N,Npadding//2:Npadding//2+N] = bfp_field*np.exp(2*1j*np.pi*n1*0.*np.sqrt(1-(np.sin(th1))**2)/0.617)
bfp_field = bfp_field_

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
meeeesh = plt.pcolormesh(bfp_field)
cb = plt.colorbar(meeeesh, pad=0.15, label='Photon number')

In [ ]:
im_field = np.fft.fftshift(np.fft.fft2(bfp_field))
psf1 = np.real(im_field)**2 + np.imag(im_field)**2

In [ ]:
def gauss(coords, A, mu1, mu2, s, c):
    x, y = coords
    return A*(1/(2*np.pi*(s**2)))*np.exp(-0.5*(((x-mu1)**2+(y-mu2)**2)/((s**2)))) + c

def fit_psf_in_pixel(data_raw):
    uuuu, vvvv = np.meshgrid(np.arange(data_raw.shape[0]),np.arange((data_raw.shape[0])))
    xdata = np.vstack((uuuu.ravel(), vvvv.ravel()))
    zdata = data_raw.ravel()
    p, pcov = curve_fit(gauss, xdata, zdata, p0=(100, 20, 20, 3, 0))
    return p, gauss((uuuu, vvvv), p[0], p[1], p[2], p[3], p[4])

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,2)
NN = len(psf1)
to_fit = psf1[NN//2-21:NN//2+22,NN//2-21:NN//2+22]/10000000
p, sim = fit_psf_in_pixel(to_fit)
print(p[3]*10*l_pixel, 0.26*(lambd/NA))
vmax2 = np.max(sim)
vmax = np.max(to_fit)
me = ax[0].imshow(to_fit, norm=LogNorm())
ax[0].plot(21+((0.61)*lambd/(10*l_pixel*NA))*np.cos(np.linspace(0, 2*np.pi, 100)), 21+((0.61)*lambd/(10*l_pixel*NA))*np.sin(np.linspace(0, 2*np.pi, 100)), c='r')
cb = plt.colorbar(me, pad=0.15, label='Photon number')
mesh2 = ax[1].imshow(sim, vmin=0, vmax=vmax2)
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

In [ ]:
ree = []
for uu, fi in enumerate([Ex[0], Ex[1], Ex[2], Ey[0], Ey[1], Ey[2]]):
    fi_n = np.zeros((N+Npadding, N+Npadding), dtype='float64')
    fi_n[Npadding//2:Npadding//2+N,Npadding//2:Npadding//2+N] = (fi)*np.exp(2*1j*np.pi*n1*0.*np.sqrt(1-(np.sin(th1))**2)/0.617)
    fi_n[np.isnan(fi_n)] = 0.
    ree.append(fi_n)
    #plt.imshow(fi_n)
    #plt.show()
Ex_im = np.array([np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(ree[0]))), np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(ree[1]))), np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(ree[2])))])
Ey_im = np.array([np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(ree[3]))), np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(ree[4]))), np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(ree[5])))])
#Ex_im = np.array([np.fft.fft2(ree[0]), np.fft.fft2(ree[1]), np.fft.fft2(-1j*ree[2])])
#Ey_im = np.array([np.fft.fft2(ree[3]), np.fft.fft2(ree[4]), np.fft.fft2(-1j*ree[5])])
Ex_pad = np.array([ree[0], ree[1], ree[2]])
Ey_pad = np.array([ree[3], ree[4], ree[5]])

In [ ]:
Mx = np.einsum('abc, ubc -> aubc', np.conj(Ex_im), Ex_im)
#Mx = Mx*np.array([[np.ones(Mx.shape[2:]),np.ones(Mx.shape[2:]),np.ones(Mx.shape[2:])],[np.ones(Mx.shape[2:]),np.ones(Mx.shape[2:]),np.ones(Mx.shape[2:])],[1j*np.ones(Mx.shape[2:]),1j*np.ones(Mx.shape[2:]),np.ones(Mx.shape[2:])]])
My = np.einsum('abc, ubc -> aubc', np.conj(Ey_im), Ey_im)
print(Mx.shape)

In [ ]:
integral1 = np.einsum('ab, bc -> ac', np.einsum('ab, bc -> ac', R, np.diag(lam)), R.T)
integral2 = np.sum(np.array([lam[i]*np.einsum('a, b -> ab', R.T[i], R.T[i]) for i in range(3)]), axis=0)
print(integral1-integral2)

from scipy.signal import fftconvolve
for i in range(3):
    for j in range(3):
        print(j)
        Mx[i,j] = np.fft.fftshift(np.fft.fft2(fftconvolve(np.flip(np.conj(Ex_pad[i]), axis=(0,1)), Ex_pad[j], mode='same')))
        My[i,j] = np.fft.fftshift(np.fft.fft2(fftconvolve(np.flip(np.conj(Ey_pad[i]), axis=(0,1)), Ey_pad[j], mode='same')))

In [ ]:
mxy = np.imag(Mx[2,1]-Mx[1,2])#(2*np.real(Ex_im[2]*np.conj(Ex_im[0])))

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
aa = plt.pcolormesh((mxy))#, norm=LogNorm())
plt.xlim((mxy.shape[0]//2-30, mxy.shape[0]//2+30))
plt.ylim((mxy.shape[0]//2-30, mxy.shape[0]//2+30))
cb = plt.colorbar(aa, pad=0.15, label='Photon number')

In [ ]:
psfx = (np.real(np.einsum('a, auv -> uv', lam, np.moveaxis(np.diagonal(np.einsum('ab, bcuv -> acuv', R.T, np.einsum('abuv, bc -> acuv', Mx, R)), axis1=0, axis2=1), -1, 0))))
psfy = np.real(np.einsum('a, auv -> uv', lam, np.moveaxis(np.diagonal(np.einsum('ab, bcuv -> acuv', R.T, np.einsum('abuv, bc -> acuv', My, R)), axis1=0, axis2=1), -1, 0)))
'''
psfx = np.zeros(Ex_im[0].shape, dtype='complex128')
for hhh in range(3):
    first_sum = np.einsum('a, abcd->bcd', R[hhh], Mx)
    psfx += lam[hhh]*np.einsum('abc, a -> bc', first_sum, R[hhh])
psfx = np.real(psfx)

psfy = np.zeros(Ex_im[0].shape, dtype='complex128')
for hhh in range(3):
    first_sum = np.einsum('a, abcd->bcd', R[hhh], My)
    psfy += lam[hhh]*np.einsum('abc, a -> bc', first_sum, R[hhh])
psfy = np.real(psfy)'''

In [ ]:
psf2 = psfx+psfy

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,2)
NN = len(psf2)
to_fit = psf2[NN//2-21:NN//2+22,NN//2-21:NN//2+22]/np.sum(psf2[NN//2-21:NN//2+22,NN//2-21:NN//2+22])
p, sim = fit_psf_in_pixel(to_fit)
print(p[3]*10*l_pixel, 0.26*(lambd/NA))
vmax2 = np.max(sim)
vmax = np.max(to_fit)
print(vmax)
aa, bb = np.meshgrid(np.arange(to_fit.shape[0])*l_pixel, np.arange(to_fit.shape[0])*l_pixel)
me = ax[0].pcolormesh(aa, bb, to_fit, norm=LogNorm(vmin=1e-6, vmax=0.028))
cb = plt.colorbar(me, pad=0.15, label='Photon number')
mesh2 = ax[1].imshow(sim)#, norm=LogNorm())
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')
#ax[0].plot(l_pixel*(21+((1*0.61)*lambd/(10*l_pixel*NA))*np.cos(np.linspace(0, 2*np.pi, 100))), l_pixel*(21+((1*0.61)*lambd/(10*l_pixel*NA))*np.sin(np.linspace(0, 2*np.pi, 100))), c='r')
ax[0].set_xlabel('x_camera ($\mu$m)')
ax[0].set_ylabel('y_camera ($\mu$m)')

In [ ]:
scalar = np.array([5.75247137e-05, 6.87539038e-05, 4.08164916e-05, 4.41025338e-06,
       1.29890657e-05, 8.22916515e-05, 1.56881803e-04, 1.55371646e-04,
       6.72128436e-05, 3.86801472e-07, 9.96100110e-05, 3.78341612e-04,
       6.31061410e-04, 5.77383917e-04, 2.02453288e-04, 3.32463238e-05,
       1.04090992e-03, 4.08718316e-03, 9.20578174e-03, 1.52327794e-02,
       2.01512732e-02, 2.20513348e-02, 2.01512732e-02, 1.52327794e-02,
       9.20578174e-03, 4.08718316e-03, 1.04090992e-03, 3.32463238e-05,
       2.02453288e-04, 5.77383917e-04, 6.31061410e-04, 3.78341612e-04,
       9.96100110e-05, 3.86801472e-07, 6.72128436e-05, 1.55371646e-04,
       1.56881803e-04, 8.22916515e-05, 1.29890657e-05, 4.41025338e-06,
       4.08164916e-05, 6.87539038e-05, 5.75247137e-05])

In [ ]:
no_rim = np.array([4.06926329e-05, 4.12862097e-05, 3.72773738e-05, 3.98402774e-05,
       5.70649655e-05, 8.22824681e-05, 9.75436957e-05, 9.44600292e-05,
       9.30452042e-05, 1.32393538e-04, 2.29926980e-04, 3.44996559e-04,
       4.00130284e-04, 3.79279812e-04, 4.49354920e-04, 9.96040719e-04,
       2.48780205e-03, 5.19104965e-03, 8.88828162e-03, 1.27987135e-02,
       1.58102124e-02, 1.69438265e-02, 1.58102124e-02, 1.27987135e-02,
       8.88828162e-03, 5.19104965e-03, 2.48780205e-03, 9.96040719e-04,
       4.49354920e-04, 3.79279812e-04, 4.00130284e-04, 3.44996559e-04,
       2.29926980e-04, 1.32393538e-04, 9.30452042e-05, 9.44600292e-05,
       9.75436957e-05, 8.22824681e-05, 5.70649655e-05, 3.98402774e-05,
       3.72773738e-05, 4.12862097e-05, 4.06926329e-05])

In [ ]:
plt.rcParams['figure.figsize'] = [15, 4]
med = (to_fit[22]+ to_fit[23])/2
plt.plot(np.arange(len(med))*l_pixel, med, label='Vectorial with RIM', c='b')
plt.plot(np.arange(len(med))*l_pixel, scalar, label='Scalar', c='r')
plt.plot(np.arange(len(med))*l_pixel, no_rim, label='Vectorial with no RIM', c='g')
plt.grid()
plt.xlim((25, 150))
plt.legend()
plt.xlabel('x_camera ($\mu$m)')
plt.ylabel('Intensity (normalized)')

In [ ]:
full_bfpx = np.zeros(Ex_im[0].shape)

first_sum = np.einsum('ab, auv -> buv', R, Ex_im)
v3X= np.einsum('a, auv -> uv', lam, np.conj(first_sum)*first_sum)

first_sum = np.einsum('ab, auv -> buv', R, Ey_im)
v3Y = np.einsum('a, auv -> uv', lam, np.conj(first_sum)*first_sum)

In [ ]:
((np.einsum('ab, auv -> buv', R, Ex_im) == np.einsum('ab, buv -> auv', R, Ex_im))==True).all()

In [ ]:
psf3 = np.real(v3X) #+v3Y

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,2)
NN = len(psf3)
to_fit = psf3[NN//2-21:NN//2+22,NN//2-21:NN//2+22]/100000000
p, sim = fit_psf_in_pixel(to_fit)
print(p[3]*10*l_pixel, 0.26*(lambd/NA))
vmax2 = np.max(sim)
vmax = np.max(to_fit)
me = ax[0].imshow(to_fit, norm=LogNorm())
cb = plt.colorbar(me, pad=0.15, label='Photon number')
mesh2 = ax[1].imshow(sim)#, norm=LogNorm())
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

In [ ]:
psf4 = np.real(np.einsum('auv, auv -> uv', Ex_im, np.einsum('ab, buv -> auv', integral1, np.conj(Ex_im))))

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,2)
NN = len(psf4)
to_fit = psf4[NN//2-21:NN//2+22,NN//2-21:NN//2+22]/100000000
p, sim = fit_psf_in_pixel(to_fit)
print(p[3]*10*l_pixel, 0.26*(lambd/NA))
vmax2 = np.max(sim)
vmax = np.max(to_fit)
me = ax[0].imshow(to_fit, norm=LogNorm())
cb = plt.colorbar(me, pad=0.15, label='Photon number')
mesh2 = ax[1].imshow(sim)#, norm=LogNorm())
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

In [ ]:
R

In [ ]:
lam

In [ ]:
field = Ex0 + Ex1+Ex2

In [ ]:
intensity = Ex0*np.conj(Ex0) + Ex1*np.conj(Ex1) + Ex2*np.conj(Ex2)

In [ ]:
version1 = np.zeros((N+Npadding, N+Npadding), dtype='float64')
version1[Npadding//2:Npadding//2+N,Npadding//2:Npadding//2+N] = (field/np.sqrt(np.cos(th1)))*np.exp(2*1j*np.pi*n1*0.*np.sqrt(1-(np.sin(th1))**2)/0.617)

In [ ]:
plt.imshow(version1)

In [ ]:
P_dipole3_x = -np.sin(eta)*np.cos(rho)*np.sqrt(lam[2])
P_dipole3_y = -np.sin(eta)*np.sin(rho)*np.sqrt(lam[2])
P_dipole3_z = np.cos(eta)*np.sqrt(lam[2])

P_dipole2_x = -np.cos(eta)*np.cos(rho)*np.sqrt(lam[1])
P_dipole2_y = -np.cos(eta)*np.sin(rho)*np.sqrt(lam[1])
P_dipole2_z = -np.sin(eta)*np.sqrt(lam[1])

P_dipole1_x = -np.sin(rho)*np.sqrt(lam[0])
P_dipole1_y = np.cos(rho)*np.sqrt(lam[0])
P_dipole1_z = 0

In [ ]:
Ex1BFP = Ex_pad[0]*P_dipole1_x + Ex_pad[1]*P_dipole1_y + Ex_pad[2]*P_dipole1_z
Ex2BFP = Ex_pad[0]*P_dipole2_x + Ex_pad[1]*P_dipole2_y + Ex_pad[2]*P_dipole2_z
Ex3BFP = Ex_pad[0]*P_dipole3_x + Ex_pad[1]*P_dipole3_y + Ex_pad[2]*P_dipole3_z

In [ ]:
Ifftx = np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Ex1BFP))))**2 + np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Ex2BFP))))**2 + np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(Ex3BFP))))**2

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,2)
NN = len(Ifftx)
to_fit = Ifftx[NN//2-21:NN//2+22,NN//2-21:NN//2+22]/100000000
p, sim = fit_psf_in_pixel(to_fit)
print(p[3]*10*l_pixel, 0.26*(lambd/NA))
vmax2 = np.max(sim)
vmax = np.max(to_fit)
me = ax[0].imshow(to_fit, norm=LogNorm())
cb = plt.colorbar(me, pad=0.15, label='Photon number')
mesh2 = ax[1].imshow(sim)#, norm=LogNorm())
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

# Basis function version

In [ ]:
# u and v are meshgrids corresponding to the image plane coordinates
# M --> see theoretical description

# THIS PART IS DEPENDANT ONTHE SPATIAL COORDINATES OF THE EMITTER AND THE FOCUS POSITION BUT NOT THE ORIENTATION

In [ ]:
u, v, M = compute_M(xp=xp, yp=yp, zp=z, d=d_, x=x, y=y, th1=th1, phi=phi, Ex0=Ex0, Ex1=Ex1, Ex2=Ex2
                    , Ey0=Ey0, Ey1=Ey1, Ey2=Ey2, r=r, r_cut=r_cut, k=k, f_o=f_o, N=N, l_pixel=l_pixel, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, MAG=MAG,
                   )#aberrations=True, defocus_coef=1e-5, spherical_coef=-1.5)

In [ ]:
M.shape

In [ ]:
M.dtype

In [ ]:
# HERE COMES THE ORIENTATION

In [ ]:
psf = PSF(rho=rho, eta=eta, delta=delta, M=M, N_photons=N_photons)

In [ ]:
psf.shape

In [ ]:
# the result is of dimension 3 (polar, x_image, y_image) or 4 if rho ... are tensors of dimension N (several psf computed)
# then the dimension is () (N, polar, x_image, y_image)

In [ ]:
x_lim = 100
plt.rcParams['figure.figsize'] = [25, 5]
fig, ax = plt.subplots(1,3)

if isinstance(psf, torch.Tensor):
    if len(psf.shape)==3:
        psfx = psf[0].detach().numpy()
        psfy = psf[1].detach().numpy()
    elif len(psf.shape)==4:
        psfx = psf[0,0].detach().numpy()
        psfy = psf[0,1].detach().numpy()
    else:
        psfx = psf[0,0,0].detach().numpy()
        psfy = psf[0,0,1].detach().numpy()
    if xp.ndim!=0:
        xpp = xp[0]
        ypp = yp[0]
    psf_unpolarized = psfx+psfy
else:
    xpp = xp
    ypp = yp
    psfx = psf[0]
    psfy = psf[1]
    psf_unpolarized = psfx+psfy

if version=='numpy':
    maag = 0.001*(MAG*mag)
    r_circle = (0.61*lambd/NA)*maag
else:
    maag = 0.001*(MAG*mag).detach().numpy()
    r_circle = (0.61*lambd/NA).detach().numpy()*maag
print(r_circle)

mesh = ax[0].pcolormesh(u, v, psfx, cmap='gray')
ax[0].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[0].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[0].set_xlabel('x_camera ($\\mu$m)')
ax[0].set_ylabel('y_camera ($\\mu$m)')
ax[0].set_title('x polarization')
secax = ax[0].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[0].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh, pad=0.15, label='Photon number')

mesh1 = ax[1].pcolormesh(u, v, psfy, cmap='gray')
ax[1].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[1].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[1].set_xlabel('x_camera ($\\mu$m)')
ax[1].set_ylabel('y_camera ($\\mu$m)')
ax[1].set_title('y polarization')
secax = ax[1].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[1].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')

mesh2 = ax[2].pcolormesh(u, v, psf_unpolarized, cmap='gray')#, norm=LogNorm())
ax[2].plot(r_circle*np.cos(np.linspace(0,2*np.pi,100)), r_circle*np.sin(np.linspace(0,2*np.pi,100)), c='r')
ax[2].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[2].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[2].set_xlabel('x_camera ($\\mu$m)')
ax[2].set_ylabel('y_camera ($\\mu$m)')
ax[2].set_title('unpolarized')
secax = ax[2].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[2].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

In [ ]:
fig, ax = plt.subplots(1,2)
NN = len(psf_unpolarized)
to_fit = psf_unpolarized[NN//2-11:NN//2+12,NN//2-11:NN//2+12]
p, sim = fit_psf_in_pixel(to_fit)
print(p[3]*120, (0.51/2.355)*(lambd/NA))
vmax2 = np.max(sim)
vmax = np.max(to_fit)
me = ax[0].imshow(to_fit)
cb = plt.colorbar(me, pad=0.15, label='Photon number')
mesh2 = ax[1].imshow(sim, vmin=0, vmax=vmax2)
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

# Noise

In [ ]:
psf_0_noisy = noise(psfx, QE=1, EM=1, b=5., sigma_b=2., sigma_r=6., bias=12.)
psf_1_noisy = noise(psfy, QE=1, EM=1, b=5., sigma_b=2., sigma_r=6., bias=12.)

x_lim = 150
plt.rcParams['figure.figsize'] = [25, 5]
fig, ax = plt.subplots(1,3)

if isinstance(psf_0_noisy, torch.Tensor):
    psf_0_noisy = psf_0_noisy.detach().numpy()
    psf_1_noisy = psf_1_noisy.detach().numpy()

mesh = ax[0].pcolormesh(u, v, psf_0_noisy, cmap='gray')
ax[0].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[0].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[0].set_xlabel('x_camera ($\\mu$m)')
ax[0].set_ylabel('y_camera ($\\mu$m)')
ax[0].set_title('x polarization')
secax = ax[0].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[0].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh, pad=0.15, label='Photon number')

mesh1 = ax[1].pcolormesh(u, v, psf_1_noisy, cmap='gray')
ax[1].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[1].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[1].set_xlabel('x_camera ($\\mu$m)')
ax[1].set_ylabel('y_camera ($\\mu$m)')
ax[1].set_title('y polarization')
secax = ax[1].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[1].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')

mesh2 = ax[2].pcolormesh(u, v, psf_0_noisy+psf_1_noisy, cmap='gray')
ax[2].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[2].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[2].set_xlabel('x_camera ($\\mu$m)')
ax[2].set_ylabel('y_camera ($\\mu$m)')
ax[2].set_title('unpolarized')
secax = ax[2].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[2].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

# gaussian fit

In [ ]:
def gauss(coords, A, mu1, mu2, s, c):
    x, y = coords
    return (16**2)*A*(1/(2*np.pi*(s**2)))*np.exp(-0.5*(((x-mu1)**2+(y-mu2)**2)/((s**2)))) + c

In [ ]:
np.sum(psf_unpolarized)

In [ ]:
X = u.detach().numpy()
Y = v.detach().numpy()

In [ ]:
xdata = np.vstack((X.ravel(), Y.ravel()))
zdata = psf_unpolarized.ravel()

In [ ]:
p, pcov = curve_fit(gauss, xdata, zdata, p0=(1000, 15, 15, 35, 2))

In [ ]:
p

In [ ]:
sim = gauss((X, Y), p[0], p[1], p[2], p[3], p[4])

In [ ]:
print(np.sum(sim[sim.shape[0]//2-6:sim.shape[0]//2+7,sim.shape[0]//2-6:sim.shape[0]//2+7]))
print(np.sum(sim))
print(np.sum(psf_unpolarized[sim.shape[0]//2-6:sim.shape[0]//2+7,sim.shape[0]//2-6:sim.shape[0]//2+7]))
print(np.sum(psf_unpolarized))

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
mee = plt.pcolormesh(u.detach().numpy(), v.detach().numpy(), sim)
plt.xlim((-200,200))
plt.ylim((-200,200))
cb = plt.colorbar(mee, pad=0.15, label='Photon number')
print(np.sum(sim))

# Several planes

## Needs to be in "several psf mode" 

In [ ]:
second_plane = [-0.8, 0.4]

In [ ]:
u, v, M = compute_M(xp=xp, yp=yp, zp=z, d=d_, x=x, y=y, th1=th1, phi=phi, Ex0=Ex0, Ex1=Ex1, Ex2=Ex2
                    , Ey0=Ey0, Ey1=Ey1, Ey2=Ey2, r=r, r_cut=r_cut, k=k, f_o=f_o, second_plane=second_plane, N=N, l_pixel=l_pixel, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, MAG=MAG,
                   )#aberrations=True, defocus_coef=1.5, spherical_coef=1e-5)

In [ ]:
M.shape

In [ ]:
psf = PSF(rho=rho, eta=eta, delta=delta, M=M, N_photons=torch.tensor([3000.]))

In [ ]:
psf.shape

In [ ]:
torch.sum(psf[0])

The first dimension is the number of the psf, the second is the plane, the third is the polarization projection and the two others are x, y

if second_plane=None, there is only one plane simulated and the polarization projection is 0/90, and the result is only of dimension 4 (cf previous part)

if second_plane = [a], then two plane are simulated. The first is the principal (in d), and the second corresponds to d+a (warning the convention for d is to take it negative, and thus same for a). The principal plane is projected on 45/135 and the second on 0, 90.

if second_plane = [a, b], then 3 plane are simulated. In the order of the array, (d+b, d, d+a), with projections (0, 45, 0.)

In [ ]:
# choose the plane you want to image
plane_nb = 1

In [ ]:
x_lim = 200
plt.rcParams['figure.figsize'] = [25, 5]
fig, ax = plt.subplots(1,3)

psfx = psf[0,plane_nb ,0].detach().numpy()
psfy = psf[0,plane_nb ,1].detach().numpy()
xpp = xp[0]
ypp = yp[0]
psf_unpolarized = psfx+psfy


mesh = ax[0].pcolormesh(u, v, psfx, cmap='gray')
ax[0].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[0].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[0].set_xlabel('x_camera ($\\mu$m)')
ax[0].set_ylabel('y_camera ($\\mu$m)')
ax[0].set_title('x polarization')
secax = ax[0].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[0].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh, pad=0.15, label='Photon number')

mesh1 = ax[1].pcolormesh(u, v, psfy, cmap='gray')
ax[1].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[1].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[1].set_xlabel('x_camera ($\\mu$m)')
ax[1].set_ylabel('y_camera ($\\mu$m)')
ax[1].set_title('y polarization')
secax = ax[1].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[1].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')

mesh2 = ax[2].pcolormesh(u, v, psf_unpolarized, cmap='gray')
ax[2].set_xlim((xpp*mag*MAG-x_lim, xpp*mag*MAG+x_lim))
ax[2].set_ylim((ypp*mag*MAG-x_lim, ypp*mag*MAG+x_lim))
ax[2].set_xlabel('x_camera ($\\mu$m)')
ax[2].set_ylabel('y_camera ($\\mu$m)')
ax[2].set_title('unpolarized')
secax = ax[2].secondary_xaxis('top', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax2 = ax[2].secondary_yaxis('right', functions=(lambda x : 1000*x/(mag*MAG), lambda x : 0.001*mag*MAG*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

# Focus

In [ ]:
nn = 150

In [ ]:
z_list = np.linspace(0.,100.,100)

In [ ]:
def fitting(xx, xx0, a, off):
    return a*(xx-xx0)**2+off

In [ ]:
def Gaussian2D_centered(xy, sigma, A):
    x, y = xy
    return A*np.exp(-(x**2)/(2*sigma**2) - (y**2)/(2*sigma**2))

In [ ]:
xi_list = []
for zz in tqdm(z_list):
    xp = torch.tensor([0. for i in range(nn)])
    yp = torch.tensor([0. for i in range(nn)])
    zp = torch.tensor([zz for i in range(nn)])
    rho = torch.tensor([50. for i in range(nn)])
    eta = torch.tensor([80. for i in range(nn)])
    delta = torch.tensor([160. for i in range(nn)])
    
    d = torch.tensor(np.linspace(-zz, -2*zz, nn))
    
    u, v, M = compute_M(xp=xp, yp=yp, zp=zp, d=d, x=x, y=y, th1=th1, phi=phi, Ex0=Ex0, Ex1=Ex1, Ex2=Ex2
                    , Ey0=Ey0, Ey1=Ey1, Ey2=Ey2, r=r, r_cut=r_cut, k=k, f_o=f_o, N=N, l_pixel=l_pixel, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, MAG=MAG)
    psf_ = PSF(rho=rho, eta=eta, delta=delta, M=M, N_photons=N_photons).detach().numpy()
    psf_ = np.sum(psf_, axis=1)

    n = int(psf_.shape[-1]/2)
    index = np.argmax(np.sum(psf_[:,n-1:n+2, n-1:n+2], axis=(1,2)))    

    xi_list.append(zp.detach().numpy()[0]/(-d.detach().numpy()[index])) #int(p[0])

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
plt.scatter(z_list, xi_list)

In [ ]:
#z_lunit = [1,2,3,4,5,6,7,8,9,10,20,50,70,120]
#z = [0.617, 1.2340, 1.8510, 2.4680, 3.0850, 3.7020, 4.3190, 4.9360, 5.5530, 6.1700, 12.3400, 30.8500, 43.1900, 74.0400, 100.]
#xi_moi = [0.5896, 0.6385, 0.6683, 0.7135, 0.726446, 0.7413, 0.7347, 0.7569, 0.7418, 0.7515, 0.7637, 0.8121, 0.8173, 0.8286, 0.8355]
#xi_yan = [0.6230, 0.6431, 0.6500, 0.6701, 0.6689, 0.6796, 0.6874, 0.6933, 0.6900, 0.6952, 0.7128, 0.7247, 0.7331, 0.7398, 0.7433]
z = np.linspace(0.,100.,100)[1:]
xi_moi = [0.6234309623430963, 0.680365296803653, 0.6930232558139535, 0.7303921568627451, 0.7450000000000001, 0.7525252525252525, 0.7680412371134021, 0.7563451776649746, 0.7525252525252525, 0.772020725388601, 0.7883597883597884, 0.7680412371134021, 0.7925531914893618, 0.7883597883597885, 0.7801047120418848, 0.8054054054054054, 0.8010752688172043, 0.7801047120418848, 0.8010752688172044, 0.8010752688172043, 0.7883597883597883, 0.7842105263157895, 0.8142076502732241, 0.8054054054054054, 0.8097826086956522, 0.8054054054054054, 0.8010752688172043, 0.8232044198895028, 0.8142076502732241, 0.814207650273224, 0.8097826086956522, 0.8277777777777778, 0.8186813186813187, 0.8097826086956521, 0.8097826086956521, 0.8054054054054054, 0.8232044198895028, 0.8232044198895027, 0.8277777777777778, 0.8277777777777778, 0.8277777777777777, 0.8232044198895028, 0.8232044198895028, 0.8186813186813187, 0.8097826086956522, 0.8142076502732241, 0.8097826086956522, 0.8277777777777778, 0.8232044198895028, 0.8277777777777778, 0.8097826086956521, 0.8277777777777777, 0.8232044198895027, 0.8232044198895028, 0.8186813186813188, 0.8232044198895028, 0.8277777777777777, 0.8232044198895029, 0.8418079096045198, 0.8465909090909092, 0.8324022346368715, 0.8277777777777777, 0.8232044198895028, 0.8370786516853933, 0.8232044198895028, 0.8324022346368715, 0.8324022346368716, 0.8277777777777778, 0.8277777777777778, 0.8277777777777778, 0.8277777777777777, 0.8514285714285714, 0.8232044198895028, 0.8277777777777778, 0.8232044198895028, 0.8277777777777778, 0.8418079096045198, 0.8370786516853933, 0.8418079096045198, 0.8418079096045197, 0.8418079096045198, 0.8370786516853932, 0.8418079096045198, 0.8370786516853933, 0.8186813186813187, 0.8232044198895028, 0.8324022346368715, 0.8324022346368716, 0.8277777777777777, 0.8324022346368715, 0.8324022346368715, 0.8277777777777778, 0.8277777777777777, 0.8418079096045198, 0.8370786516853933, 0.8277777777777778, 0.8465909090909091, 0.8465909090909091, 0.8418079096045198]
xi_yan = [0.6394849785407726, 0.6592920353982301, 0.6772727272727272, 0.6834862385321101, 0.6930232558139534, 0.6962616822429906, 0.7028301886792453, 0.7028301886792453, 0.7061611374407584, 0.7095238095238094, 0.7129186602870814, 0.7163461538461539, 0.7129186602870813, 0.7163461538461539, 0.7198067632850241, 0.7233009708737864, 0.7233009708737863, 0.7198067632850241, 0.7163461538461537, 0.7303921568627452, 0.7268292682926829, 0.726829268292683, 0.726829268292683, 0.7233009708737863, 0.7268292682926829, 0.7303921568627451, 0.7303921568627452, 0.7339901477832512, 0.7303921568627451, 0.7268292682926829, 0.7376237623762375, 0.7339901477832512, 0.7303921568627451, 0.7268292682926829, 0.7339901477832512, 0.7303921568627451, 0.7376237623762376, 0.7339901477832512, 0.7376237623762376, 0.7339901477832512, 0.7339901477832512, 0.7339901477832512, 0.7339901477832512, 0.7376237623762376, 0.7339901477832512, 0.7412935323383085, 0.745, 0.7412935323383084, 0.7412935323383084, 0.7412935323383084, 0.7376237623762376, 0.7376237623762377, 0.7339901477832513, 0.7339901477832512, 0.7376237623762376, 0.7450000000000001, 0.7412935323383084, 0.7412935323383085, 0.7412935323383085, 0.7376237623762377, 0.7412935323383084, 0.7376237623762375, 0.745, 0.745, 0.7412935323383085, 0.7412935323383085, 0.7376237623762376, 0.7376237623762377, 0.745, 0.7412935323383084, 0.745, 0.7412935323383085, 0.7412935323383084, 0.7412935323383084, 0.7450000000000001, 0.745, 0.7412935323383084, 0.745, 0.745, 0.7450000000000001, 0.745, 0.745, 0.745, 0.745, 0.7412935323383085, 0.7412935323383084, 0.7412935323383084, 0.7412935323383084, 0.7412935323383084, 0.745, 0.745, 0.7487437185929648, 0.7487437185929647, 0.745, 0.7487437185929648, 0.745, 0.745, 0.745, 0.7487437185929647]

In [ ]:
n2/n1

In [ ]:
def fit(x, a, b, c):
    return a - (b/x)**c

In [ ]:
pp, ppcov = curve_fit(fit, np.array(z), xi_moi, p0=(0.8, 10, 0.2))

In [ ]:
pp

In [ ]:
epsilon = -(0.617)/(4*z*n2)

In [ ]:
mm = np.sqrt(n1**2-n2**2)

In [ ]:
xi_univ = (n2/n1)*(1-epsilon-(mm/n1)*np.sqrt(epsilon*(epsilon-2)))/(1-((n2/n1)**2)*epsilon*(2-epsilon))

In [ ]:
plt.rcParams['figure.figsize'] = [5, 3]
plt.scatter(z, np.array(xi_list)[:-1], marker='x', label='Simulated data')
plt.plot(z, xi_univ, c='r', label='Loginov $\\mathit{et}$ $\\mathit{al.}$ (2024) (analytical)')
#plt.plot(z, fit(np.array(z), pp[0], pp[1], pp[2]))
plt.scatter(z, xi_yan, label='Yan $\\mathit{et}$ $\\mathit{al.}$ (2019) (simulated)')
#plt.plot(z, [n2/n1 for k in range(len(z))], label='Geometrical optics', c='r')
plt.xlabel('Particule-interface distance z ($\\mu m$)')
plt.ylabel('$\\zeta = z/z_A$')
plt.xlim((0,101))
plt.ylim((0.5,0.87))
#plt.xlim((0,10))
plt.legend(loc='lower right')
plt.grid()

In [ ]:
np.arcsin(1.4/1.52)*180/np.pi